# Support vector machine

*Source: Project assignment form **Andrew Ng** Machine Learning [course](https://www.coursera.org/learn/machine-learning) (Coursera).*

## About the data set

This database was created to identify a voice as male or female, based upon acoustic properties of the voice and speech. The dataset consists of 3,168 recorded voice samples, collected from male and female speakers. The voice samples are pre-processed by acoustic analysis in R using the seewave and tuneR packages, with an analyzed frequency range of 0hz-280hz ([human vocal range](https://en.wikipedia.org/wiki/Voice_frequency#Fundamental_frequency))

## Data set dictionary

The following acoustic properties of each voice are measured and included within the CSV:

- `meanfreq`: mean frequency (in kHz)
- `sd`: standard deviation of frequency
- `median`: median frequency (in kHz)
- `Q25`: first quantile (in kHz)
- `Q75`: third quantile (in kHz)
- `IQR`: interquantile range (in kHz)
- `skew`: skewness (see note in specprop description)
- `kurt`: kurtosis (see note in specprop description)
- `sp.ent`: spectral entropy
- `sfm`: spectral flatness
- `mode`: mode frequency
- `centroid`: frequency centroid (see specprop)
- `peakf`: peak frequency (frequency with highest energy)
- `meanfun`: average of fundamental frequency measured across acoustic signal
- `minfun`: minimum fundamental frequency measured across acoustic signal
- `maxfun`: maximum fundamental frequency measured across acoustic signal
- `meandom`: average of dominant frequency measured across acoustic signal
- `mindom`: minimum of dominant frequency measured across acoustic signal
- `maxdom`: maximum of dominant frequency measured across acoustic signal
- `dfrange`: range of dominant frequency measured across acoustic signal
- `modindx`: modulation index. Calculated as the accumulated absolute difference between adjacent measurements of fundamental frequencies divided by the frequency range
- `label`: male or female

## Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Reading CSV into Pandas DataFrame

In [2]:
df = pd.read_csv('../data/voice.csv')
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


## Correlation table between features

In [3]:
df.corr()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
meanfreq,1.000000,-0.739039,0.925445,0.911416,0.740997,-0.627605,-0.322327,-0.316036,-0.601203,-0.784332,0.687715,1.000000,0.460844,0.383937,0.274004,0.536666,0.229261,0.519528,0.515570,-0.216979
sd,-0.739039,1.000000,-0.562603,-0.846931,-0.161076,0.874660,0.314597,0.346241,0.716620,0.838086,-0.529150,-0.739039,-0.466281,-0.345609,-0.129662,-0.482726,-0.357667,-0.482278,-0.475999,0.122660
median,0.925445,-0.562603,1.000000,0.774922,0.731849,-0.477352,-0.257407,-0.243382,-0.502005,-0.661690,0.677433,0.925445,0.414909,0.337602,0.251328,0.455943,0.191169,0.438919,0.435621,-0.213298
Q25,0.911416,-0.846931,0.774922,1.000000,0.477140,-0.874189,-0.319475,-0.350182,-0.648126,-0.766875,0.591277,0.911416,0.545035,0.320994,0.199841,0.467403,0.302255,0.459683,0.454394,-0.141377
Q75,0.740997,-0.161076,0.731849,0.477140,1.000000,0.009636,-0.206339,-0.148881,-0.174905,-0.378198,0.486857,0.740997,0.155091,0.258002,0.285584,0.359181,-0.023750,0.335114,0.335648,-0.216475
IQR,-0.627605,0.874660,-0.477352,-0.874189,0.009636,1.000000,0.249497,0.316185,0.640813,0.663601,-0.403764,-0.627605,-0.534462,-0.222680,-0.069588,-0.333362,-0.357037,-0.337877,-0.331563,0.041252
skew,-0.322327,0.314597,-0.257407,-0.319475,-0.206339,0.249497,1.000000,0.977020,-0.195459,0.079694,-0.434859,-0.322327,-0.167668,-0.216954,-0.080861,-0.336848,-0.061608,-0.305651,-0.304640,-0.169325
kurt,-0.316036,0.346241,-0.243382,-0.350182,-0.148881,0.316185,0.977020,1.000000,-0.127644,0.109884,-0.406722,-0.316036,-0.194560,-0.203201,-0.045667,-0.303234,-0.103313,-0.274500,-0.272729,-0.205539
sp.ent,-0.601203,0.716620,-0.502005,-0.648126,-0.174905,0.640813,-0.195459,-0.127644,1.000000,0.866411,-0.325298,-0.601203,-0.513194,-0.305826,-0.120738,-0.293562,-0.294869,-0.324253,-0.319054,0.198074
sfm,-0.784332,0.838086,-0.661690,-0.766875,-0.378198,0.663601,0.079694,0.109884,0.866411,1.000000,-0.485913,-0.784332,-0.421066,-0.362100,-0.192369,-0.428442,-0.289593,-0.436649,-0.431580,0.211477


## Searching for null values

In [4]:
print(df.isnull().sum())

meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
label       0
dtype: int64


In [5]:
print(type(df.shape))
print(df.shape)
print(df.shape[0])

<class 'tuple'>
(3168, 21)
3168


**Comment:** There are 21 features and 3168 instances (observations).

In [6]:
print('Total number of labels: {}'.format(df.shape[0]))
print('Number of male labels: {}'.format(df[df.label == 'male'].shape[0]))
print('Number of female labels: {}'.format(df[df.label == 'female'].shape[0]))

Total number of labels: 3168
Number of male labels: 1584
Number of female labels: 1584


**Comment:** equal number of male and female labels.

## Separating the *features* and the *labels*

In [7]:
X = df.iloc[:, :-1]
X.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.000000,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.000000,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,0.000000,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,0.083878,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,0.104261,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274


## Converting string value to `int` type for the *labels*

In [8]:
from sklearn.preprocessing import LabelEncoder
y = df.iloc[:, -1]

# encode label category
# male -> 1
# female -> 0

gender_encoder = LabelEncoder()
y = gender_encoder.fit_transform(y)
print(y)
print(type(y))

[1 1 1 ... 0 0 0]
<class 'numpy.ndarray'>


## Data standardization

Standardization refers to shifting the distribution of each attribute (i.e. column or feature) to have a mean of zero and a standard deviation of one (unit variance). It is useful to standardize attributes for a model. Standardization of data sets is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data.

In [9]:
# scale the data to be between -1 and 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X

array([[-4.04924806,  0.4273553 , -4.22490077, ..., -1.43142165,
        -1.41913712, -1.45477229],
       [-3.84105325,  0.6116695 , -3.99929342, ..., -1.41810716,
        -1.4058184 , -1.01410294],
       [-3.46306647,  1.60384791, -4.09585052, ..., -1.42920257,
        -1.41691733, -1.06534356],
       ...,
       [-1.29877326,  2.32272355, -0.05197279, ..., -0.5992661 ,
        -0.58671739,  0.17588664],
       [-1.2452018 ,  2.012196  , -0.01772849, ..., -0.41286326,
        -0.40025537,  1.14916112],
       [-0.51474626,  2.14765111, -0.07087873, ..., -1.27608595,
        -1.2637521 ,  1.47567886]])

## Splitting the data into *training* and *testing* set for better generalization

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

## Running SVM with default *hyperparameters*

Recall that a kernel is a function that returns the result of a dot product performed in another space. More formally, we can write:

Definition: Given a mapping function $\phi : X \to V$, we call the function $K: X \to \mathbb{R}$ definded by $K(\mathbf{x}, \mathbf{x'}) = \langle\ \phi{(\mathbf{x})}, \phi{(\mathbf{x'})} \rangle_V$, where $\langle\,,\rangle_V$ denotes an inner product in $V$, a **kernel function**. 

The default kernel used by the support vector classification module of `sklearn` is the **radial basis function** (RBF) kernel. also recall that the RBF kernel on two samples $\mathbf{x}$ and $\mathbf{x'}$, represented as feature vectors in some input space, is defined as

$$￼￼K(\mathbf{x}, \mathbf{x'}) = \exp\left(-\frac{\|\mathbf{x} - \mathbf{x'}\|^2}{2\sigma^2}\right)$$

where $\|\mathbf{x} - \mathbf{x'}\|^2$ may be recognized as the squared Euclidean distance between the two feature vectors. This kernel is also called the Gaussian kernel, and correspond to an infinite dimensional feature mapping $\phi$.

In [11]:
# import support vector classification module
from sklearn.svm import SVC

from sklearn import metrics

# instantiate SVM with default hyperparameters
svc = SVC()

svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy score:')
print(metrics.accuracy_score(y_test, y_pred))

Accuracy score:
0.9763406940063092


## Hyperparameter tuning 

### Linear kernel

In [12]:
svc = SVC(kernel = 'linear')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy score:')
print(metrics.accuracy_score(y_test, y_pred))

Accuracy score:
0.9779179810725552


### Polynomial kernel

In [13]:
svc = SVC(kernel = 'poly')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy score:')
print(metrics.accuracy_score(y_test, y_pred))

Accuracy score:
0.9589905362776026


**Comment:** Polynomial kernel is performing poorly. Possible overfitting the training data set.

## K-fold cross validation 

### Linear kernel

In [14]:
from sklearn.cross_validation import cross_val_score
svc = SVC(kernel = 'linear')
scores = cross_val_score(svc, X, y, cv = 10, scoring  = 'accuracy')
print(scores)
print(scores.mean())

/Users/georgioskarakostas/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[0.91194969 0.97169811 0.97169811 0.97169811 0.96202532 0.98734177
 0.99050633 0.98101266 0.95253165 0.99367089]
0.9694132632752168


### Gaussian kernel

In [15]:
from sklearn.cross_validation import cross_val_score
svc = SVC(kernel = 'rbf')
scores = cross_val_score(svc, X, y, cv = 10, scoring = 'accuracy')
print(scores)
print(scores.mean())

[0.93396226 0.95597484 0.96855346 0.9591195  0.96835443 0.99683544
 0.98734177 0.98101266 0.91455696 0.99367089]
0.9659382214791815


### Polynomial kernel

In [16]:
from sklearn.cross_validation import cross_val_score
svc = SVC(kernel = 'poly')
scores = cross_val_score(svc, X, y, cv = 10, scoring = 'accuracy')
print(scores)
print(scores.mean())

[0.89937107 0.94654088 0.93081761 0.91509434 0.95886076 1.
 0.98101266 0.96835443 0.87974684 0.9778481 ]
0.9457646684181196
